<a href="https://colab.research.google.com/github/disha2sinha/Movie-Recommendation-System/blob/master/Content_Based_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from ast import literal_eval
import pandas as pd
import math
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import os
from sklearn.cluster import KMeans

## **IMPORTING MOVIES DATASET WHICH WAS PREVIOUSLY ARRANGED ACCORDING TO POPULARITY**

In [2]:
movies_data = pd.read_csv('https://raw.githubusercontent.com/disha2sinha/Movie-Recommendation-System/master/Revised%20Datasets/PopularMovies.csv?token=AKDEMFILGZCM33P25O3KMV3ABEYBI',engine='python',index_col=0)
movies_data.head()

,tmdbId,title,budget,profit,score,release_year,release_day,keywords,genres,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,211672,Minions,74000000,1.082731e+09,0.634881,2015,Wednesday,"['assistant', 'aftercreditsstinger', 'duringcr...",Family|Animation|Adventure|Comedy,United States of America,Universal Pictures|Illumination Entertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda
1,177572,Big Hero 6,165000000,4.871054e+08,0.598368,2014,Friday,"['brother brother relationship', 'hero', 'tale...",Adventure|Family|Animation|Action|Comedy,United States of America,Walt Disney Pictures|Walt Disney Animation Stu...,English,102.0,The special bond that develops between plus-si...,Released,Scott Adsit|Ryan Potter|Daniel Henney|T.J. Mil...,Chris Williams
2,155,The Dark Knight,185000000,8.195584e+08,0.565743,2008,Wednesday,"['dc comics', 'crime fighter', 'secret identit...",Drama|Action|Crime|Thriller,United Kingdom|United States of America,DC Comics|Legendary Pictures|Warner Bros.|DC E...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan
3,680,Pulp Fiction,8000000,2.059288e+08,0.558571,1994,Saturday,"['transporter', 'brothel', 'drug dealer', 'box...",Thriller|Crime,United States of America,Miramax Films|A Band Apart|Jersey Films,English,154.0,"A burger-loving hit man, his philosophical par...",Released,John Travolta|Samuel L. Jackson|Uma Thurman|Br...,Quentin Tarantino
4,278,The Shawshank Redemption,25000000,3.341469e+06,0.554077,1994,Friday,"['prison', 'corruption', 'police brutality', '...",Drama|Crime,United States of America,Castle Rock Entertainment|Warner Bros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,Tim Robbins|Morgan Freeman|Bob Gunton|Clancy B...,Frank Darabont


In [3]:
movies_data.shape

(9081, 17)

In [4]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9081 entries, 0 to 9080
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                9081 non-null   int64  
 1   title                 9081 non-null   object 
 2   budget                9081 non-null   int64  
 3   profit                9081 non-null   float64
 4   score                 9081 non-null   float64
 5   release_year          9081 non-null   int64  
 6   release_day           9081 non-null   object 
 7   keywords              9081 non-null   object 
 8   genres                9046 non-null   object 
 9   production_countries  8787 non-null   object 
 10  production_companies  8417 non-null   object 
 11  original_language     9079 non-null   object 
 12  runtime               9081 non-null   float64
 13  content               9069 non-null   object 
 14  status                9081 non-null   object 
 15  cast                 

In [5]:
movies_data=movies_data.replace(np.nan,'Unknown',regex=True)
movies_data['release_year']=movies_data['release_year'].replace('NaT','1874',regex=True)

In [6]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9081 entries, 0 to 9080
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tmdbId                9081 non-null   int64  
 1   title                 9081 non-null   object 
 2   budget                9081 non-null   int64  
 3   profit                9081 non-null   float64
 4   score                 9081 non-null   float64
 5   release_year          9081 non-null   int64  
 6   release_day           9081 non-null   object 
 7   keywords              9081 non-null   object 
 8   genres                9081 non-null   object 
 9   production_countries  9081 non-null   object 
 10  production_companies  9081 non-null   object 
 11  original_language     9081 non-null   object 
 12  runtime               9081 non-null   float64
 13  content               9081 non-null   object 
 14  status                9081 non-null   object 
 15  cast                 

## **KEYWORDS ARE FORMED FROM ALL THE NON-NUMERIC COLUMNS**

In [7]:
movies_data['production_companies']=movies_data['production_companies'].str.replace(" ","")
movies_data['production_companies']=movies_data['production_companies'].str.replace("|","  ")
movies_data['production_countries']=movies_data['production_countries'].str.replace(" ","")
movies_data['production_countries']=movies_data['production_countries'].str.replace("|","  ")
movies_data['cast']=movies_data['cast'].str.replace(" ","")
movies_data['cast']=movies_data['cast'].str.replace("|","  ")
movies_data['genres']=movies_data['genres'].str.replace("|"," ")
movies_data['director']=movies_data['director'].str.replace(" ","")
movies_data['keywords']=movies_data['keywords'].apply(lambda x:x[1:-1])
movies_data['keywords']=movies_data['keywords'].str.replace(',',"  ")
movies_data['keywords']=movies_data['keywords'].str.replace("'","")
movies_data.head(10)

,tmdbId,title,budget,profit,score,release_year,release_day,keywords,genres,production_countries,production_companies,original_language,runtime,content,status,cast,director
0,211672,Minions,74000000,1.082731e+09,0.634881,2015,Wednesday,assistant aftercreditsstinger duringcredit...,Family Animation Adventure Comedy,UnitedStatesofAmerica,UniversalPictures IlluminationEntertainment,English,91.0,"Minions Stuart, Kevin and Bob are recruited by...",Released,SandraBullock JonHamm MichaelKeaton Allison...,KyleBalda
1,177572,Big Hero 6,165000000,4.871054e+08,0.598368,2014,Friday,brother brother relationship hero talent ...,Adventure Family Animation Action Comedy,UnitedStatesofAmerica,WaltDisneyPictures WaltDisneyAnimationStudios,English,102.0,The special bond that develops between plus-si...,Released,ScottAdsit RyanPotter DanielHenney T.J.Mill...,ChrisWilliams
2,155,The Dark Knight,185000000,8.195584e+08,0.565743,2008,Wednesday,dc comics crime fighter secret identity ...,Drama Action Crime Thriller,UnitedKingdom UnitedStatesofAmerica,DCComics LegendaryPictures WarnerBros. DCEn...,English,152.0,Batman raises the stakes in his war on crime. ...,Released,ChristianBale MichaelCaine HeathLedger Aaro...,ChristopherNolan
3,680,Pulp Fiction,8000000,2.059288e+08,0.558571,1994,Saturday,transporter brothel drug dealer boxer ...,Thriller Crime,UnitedStatesofAmerica,MiramaxFilms ABandApart JerseyFilms,English,154.0,"A burger-loving hit man, his philosophical par...",Released,JohnTravolta SamuelL.Jackson UmaThurman Bru...,QuentinTarantino
4,278,The Shawshank Redemption,25000000,3.341469e+06,0.554077,1994,Friday,prison corruption police brutality priso...,Drama Crime,UnitedStatesofAmerica,CastleRockEntertainment WarnerBros.,English,142.0,Framed in the 1940s for the double murder of h...,Released,TimRobbins MorganFreeman BobGunton ClancyBr...,FrankDarabont
5,210577,Gone Girl,61000000,3.083304e+08,0.523202,2014,Wednesday,based on novel marriage crisis disappearan...,Mystery Thriller Drama,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,145.0,With his wife's disappearance having become th...,Released,BenAffleck RosamundPike CarrieCoon NeilPatr...,DavidFincher
6,293660,Deadpool,58000000,7.251130e+08,0.518324,2016,Tuesday,anti hero mercenary marvel comic superhe...,Action Adventure Comedy,UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation MarvelEnte...,English,108.0,Deadpool tells the origin story of former Spec...,Released,RyanReynolds MorenaBaccarin EdSkrein T.J.Mi...,TimMiller
7,157336,Interstellar,165000000,5.101200e+08,0.514384,2014,Wednesday,saving the world artificial intelligence f...,Adventure Drama Science Fiction,Canada UnitedStatesofAmerica UnitedKingdom,ParamountPictures LegendaryPictures WarnerBr...,English,169.0,Interstellar chronicles the adventures of a gr...,Released,MatthewMcConaughey JessicaChastain AnneHatha...,ChristopherNolan
8,550,Fight Club,63000000,3.785375e+07,0.495441,1999,Friday,support group dual identity nihilism rag...,Drama,Germany UnitedStatesofAmerica,TwentiethCenturyFoxFilmCorporation RegencyEnt...,English,139.0,A ticking-time-bomb insomniac and a slippery s...,Released,EdwardNorton BradPitt MeatLoaf JaredLeto H...,DavidFincher
9,118340,Guardians of the Galaxy,170000000,6.033286e+08,0.484757,2014,Wednesday,marvel comic spaceship space outer space...,Action Science Fiction Adventure,UnitedKingdom UnitedStatesofAmerica,MarvelStudios MovingPictureCompany(MPC) Bull...,English,121.0,"Light years from Earth, 26 years after being a...",Released,ChrisPratt ZoeSaldana DaveBautista VinDiese...,JamesGunn


In [8]:
movies_data['keywords']=movies_data['keywords']+" " +movies_data['cast']+" "+movies_data['production_companies']+" "+movies_data['production_countries']+" "+movies_data['genres']+" "+movies_data['original_language']+" "+movies_data['release_day']+" "+movies_data['content']+" "+movies_data['director']
movies_data['keywords']=movies_data['keywords'].replace({"[^A-Za-z0-9]+": " "},regex=True)
movies_data['keywords'][0]

'assistant aftercreditsstinger duringcreditsstinger evil mastermind minions 3d SandraBullock JonHamm MichaelKeaton AllisonJanney SteveCoogan JenniferSaunders GeoffreyRush SteveCarell PierreCoffin KatyMixon MichaelBeattie HiroyukiSanada DaveRosenbaum AlexDowding PaulThornley KyleBalda AvaAcres UniversalPictures IlluminationEntertainment UnitedStatesofAmerica Family Animation Adventure Comedy English Wednesday Minions Stuart Kevin and Bob are recruited by Scarlet Overkill a super villain who alongside her inventor husband Herb hatches a plot to take over the world Before Gru they had a history of bad bosses KyleBalda'

## **WORD VECTORIZER IS CREATED FROM THE KEYWORDS AND A NEW DATAFRAME IS FORMED COMBINING THE NUMERIC COLUMNS AND WORD VECTORIZER FOR EACH MOVIE ID**

In [9]:
vectorizer=TfidfVectorizer(analyzer='word',stop_words='english')
X=vectorizer.fit_transform(movies_data['keywords'])
tfidf_df=pd.DataFrame.sparse.from_spmatrix(X)

In [10]:
movies_df=movies_data[['tmdbId','budget','profit','score','release_year','runtime']]
movies_df=movies_df.join(tfidf_df)
movies_df.set_index('tmdbId',inplace=True)
movies_df.shape

(9081, 124699)

In [11]:
dftomat=scipy.sparse.csr_matrix(movies_df.values)

In [12]:
dftomat[1,:].toarray()

array([[1.65000000e+08, 4.87105443e+08, 5.98367835e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [13]:
dftomat.shape

(9081, 124699)

## **PERFORMING KMeans ON THE NEW DATAFRAME TO FORM CLUSTERS OF SIMILAR MOVIES:**

### **FINDING OPTIMAL K USING ELBOW METHOD**

In [14]:
#sse={}
#for k in range(4,15):
  #print("K=",k)
  #kmeans=KMeans(n_clusters=k,init='random',max_iter=100).fit(dftomat)
  #sse[k]=kmeans.inertia_
#plt.plot(list(sse.keys()),list(sse.values()),'bx-')
#plt.xlabel("Number of Cluster")
#plt.ylabel("SSE")
#plt.savefig("elbow_method.png")

### **OPTIMAL K VALUE IS 6**

In [15]:
kmeans=KMeans(n_clusters=6,init='k-means++',max_iter=500).fit(dftomat)
movies_df['KmeansCluster']=kmeans.labels_

In [16]:
print("Cluster 0: ",movies_df[movies_df['KmeansCluster']==0].KmeansCluster.count())
print("Cluster 1: ",movies_df[movies_df['KmeansCluster']==1].KmeansCluster.count())
print("Cluster 2: ",movies_df[movies_df['KmeansCluster']==2].KmeansCluster.count())
print("Cluster 3: ",movies_df[movies_df['KmeansCluster']==3].KmeansCluster.count())
print("Cluster 4: ",movies_df[movies_df['KmeansCluster']==4].KmeansCluster.count())
print("Cluster 5: ",movies_df[movies_df['KmeansCluster']==5].KmeansCluster.count())

Cluster 0:  7726
Cluster 1:  122
Cluster 2:  6
Cluster 3:  320
Cluster 4:  849
Cluster 5:  58


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim=cosine_similarity(dftomat)

In [18]:
cosine_sim=pd.DataFrame(cosine_sim,index=movies_df.index,columns=movies_df.index)
cosine_sim.head(20)

tmdbId,211672,177572,155,680,278,210577,293660,157336,550,118340,238,13,19995,27205,244786,78,11,424,129,1892,24428,122,637,137113,120,16869,497,603,1891,68718,271110,240,121,150540,73,77338,106646,245891,76203,205596,...,209112,60304,71679,331,225886,5491,7278,86834,216015,1927,14161,9738,9760,49849,2486,242512,81005,329833,1250,414,8645,1979,205321,10196,43074,87818,122081,71880,9480,71676,314,76757,14164,44912,47933,82700,415,47964,241251,166424
tmdbId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
211672,1.000000,0.966809,0.988201,0.999568,0.199758,0.991936,0.999933,0.970236,0.572282,0.978773,0.999069,0.999804,0.999702,0.985973,0.967204,0.247187,0.998550,0.999987,0.999944,0.999964,0.995058,0.999730,0.999636,0.778869,0.998716,0.978923,0.981470,0.996150,0.999433,0.973680,0.979715,0.957754,0.999694,0.983352,0.256718,0.999323,0.965950,0.977017,0.998702,0.999990,...,0.951344,0.977956,0.959069,0.967155,0.933478,-0.395174,0.907367,0.914713,0.999976,0.672395,0.963929,0.942431,0.975371,0.140179,0.866843,0.999846,0.081929,0.185968,0.778217,0.945474,0.897146,0.815618,0.999299,0.790525,0.566489,0.988024,0.993196,0.716007,0.831770,0.884495,-0.108647,0.113349,-0.495926,0.166394,0.844490,0.708574,0.720256,0.942730,0.999900,0.433691
177572,0.966809,1.000000,0.994535,0.958877,0.443479,0.991394,0.969702,0.999905,0.762812,0.998651,0.954888,0.971683,0.972755,0.995892,0.999999,0.486555,0.951655,0.968104,0.964048,0.964618,0.987401,0.972481,0.973350,0.913264,0.978510,0.998612,0.997852,0.985485,0.957661,0.999596,0.998399,0.999444,0.972831,0.997141,0.495135,0.956751,0.999994,0.999052,0.978567,0.965634,...,0.998495,0.998848,0.999587,0.999999,0.994126,-0.147354,0.984647,0.987601,0.968561,0.839197,0.999939,0.996590,0.999353,0.388504,0.965459,0.962171,0.333851,0.430839,0.912840,0.997309,0.980232,0.936377,0.956568,0.920763,0.758237,0.994654,0.989985,0.870605,0.945996,0.974341,0.148947,0.363441,-0.257598,0.412810,0.953299,0.865346,0.873593,0.996664,0.970333,0.649518
155,0.988201,0.994535,1.000000,0.983269,0.347479,0.999644,0.989908,0.995879,0.691134,0.998615,0.980674,0.991042,0.991643,0.999903,0.994696,0.392683,0.978524,0.988971,0.986522,0.986872,0.998525,0.991491,0.991973,0.865742,0.994691,0.998653,0.999238,0.997823,0.982485,0.997100,0.998849,0.990502,0.991686,0.999581,0.401721,0.981894,0.994181,0.998138,0.994719,0.987492,...,0.987313,0.998399,0.991125,0.994676,0.977394,-0.249813,0.961042,0.965815,0.989241,0.777833,0.993322,0.982530,0.997646,0.290178,0.932980,0.985357,0.233613,0.334268,0.865222,0.984203,0.954219,0.894614,0.981775,0.875000,0.686024,0.999999,0.999314,0.814483,0.906981,0.945518,0.044894,0.264190,-0.357070,0.315461,0.916557,0.808292,0.818010,0.982696,0.990272,0.566585
680,0.999568,0.958877,0.983269,1.000000,0.170858,0.987781,0.999160,0.962695,0.547920,0.972323,0.999906,0.998788,0.998553,0.980638,0.959316,0.218587,0.999701,0.999404,0.999823,0.999780,0.991708,0.998615,0.998410,0.760089,0.996795,0.972494,0.975411,0.993141,0.999991,0.966556,0.973398,0.948883,0.998535,0.977584,0.228186,0.999973,0.957924,0.970326,0.996773,0.999691,...,0.941871,0.971393,0.950327,0.959262,0.922528,-0.422016,0.894614,0.902434,0.999339,0.650337,0.955685,0.932190,0.968463,0.111002,0.851807,0.999930,0.052586,0.156994,0.759413,0.935487,0.883768,0.798251,0.999968,0.772174,0.542011,0.983059,0.989342,0.695169,0.815086,0.870393,-0.137832,0.084083,-0.521247,0.137325,0.828376,0.687517,0.699545,0.932514,0.999050,0.407006
278,0.199758,0.443479,0.347479,0.170858,1.000000,0.322329,0.211089,0.431093,0.917847,0.396334,0.157308,0.219140,0.223605,0.360499,0.442090,0.998816,0.146729,0.204769,0.189366,0.191479,0.296064,0.222456,0.226121,0.770131,0.249136,0.395659,0.383808,0.284888,0.166665,0.417827,0.392064,0.473112,0.223926,0.374479,0.998289,0.163563,0.446471,0.404033,0.249403,0.195287,...,0.491960,0.399955,0.469046,0.442262,0.537876,0.821153,0.593122,0.578681,0.206560,0.859591,0.453347,0.515918,0.410962,0.998172,0.661691,0.182514,0.992917,0.999901,0

In [47]:
def recommendations(movie_name):
  movie_idx=movies_data[movies_data['title']==movie_name]['tmdbId'].values[0]
  movie_cluster=movies_df.at[movie_idx,'KmeansCluster']
  bestRecommendationsIdx=movies_df[movies_df['KmeansCluster']==movie_cluster].index.values
  scores=[]
  for index in bestRecommendationsIdx:
    scores.append((index,cosine_sim.at[movie_idx,index]))
  scores.sort(key = lambda x: x[1],reverse=True)
  bestRecommendationsdf=pd.DataFrame(columns=['title','genres','cast','director'])
  for tup in scores:
    if tup[0]!=movie_idx:
      bestRecommendationsdf=pd.concat([bestRecommendationsdf,movies_data[movies_data['tmdbId']==tup[0]][['title','genres','cast','director']]])
  return bestRecommendationsdf.head(50)
recommendations('Minions')

,title,genres,cast,director
156,Jurassic Park,Adventure Science Fiction,SamNeill LauraDern JeffGoldblum RichardAtte...,StevenSpielberg
8983,The Twilight Saga: New Moon,Adventure Fantasy Drama Romance,KristenStewart RobertPattinson TaylorLautner...,ChrisWeitz
49,The Lion King,Family Animation Drama,JonathanTaylorThomas MatthewBroderick JamesE...,RogerAllers
6,Deadpool,Action Adventure Comedy,RyanReynolds MorenaBaccarin EdSkrein T.J.Mi...,TimMiller
261,Despicable Me 2,Animation Comedy Family,SteveCarell KristenWiig BenjaminBratt Miran...,PierreCoffin
21,The Lord of the Rings: The Return of the King,Adventure Fantasy Action,ElijahWood IanMcKellen ViggoMortensen LivTy...,PeterJackson
32,The Lord of the Rings: The Two Towers,Adventure Fantasy Action,ElijahWood IanMcKellen ViggoMortensen LivTy...,PeterJackson
8998,The Secret Life of Pets,Animation Family,LouisC.K. EricStonestreet KevinHart JennySl...,ChrisRenaud
981,Independence Day,Action Adventure Science Fiction,WillSmith BillPullman JeffGoldblum MaryMcDo...,RolandEmmerich
51,Harry Potter and the Deathly Hallows: Part 2,Family Fantasy Adventure,DanielRadcliffe RupertGrint EmmaWatson Alan...,DavidYates
